In [77]:
Min_Int = -2**31
Max_Int = 2**31-1

import ast, copy

from enum import Enum
class Translate_Actions (Enum):
    PROC_DEC      = 1
    FUNC_DEC      = 2
    FUNC_RESULT   = 3
    IN_PARAM      = 4
    OUT_PARAM     = 5
    IN_OUT_PARAM  = 6
    IN_GLOBAL     = 7
    IN_OUT_GLOBAL = 8
    LOCAL_RW      = 9
    LOCAL_RO      = 10
    EVAL          = 11
    ASSIGN        = 12
    DEFINE_CONST  = 13
    DEFER_CONST   = 14
    IF_COND       = 15
    ELSE_COND     = 16
    ELSE_BRANCH   = 17
    END_IF        = 18
    ASSIGN_EXPR   = 19
    @classmethod
    def pos (cls, act):
        return act.value - 1
        
Action = Enum('Action', ['PROC_DEC', 'FUNC_DEC', 'FUNC_RESULT', 
                         'IN_PARAM', 'OUT_PARAM', 'IN_OUT_PARAM',
                         'IN_GLOBAL', 'OUT_GLOBAL', 'IN_OUT_GLOBAL', 
                         'LOCAL_RW', 'LOCAL_RO',
                         'EVAL', 'ASSIGN', 'DEFINE_CONST', 'DEFER_CONST', 
                         'IF_COND', 'ELSE_COND', 'ELSE_BRANCH', 'END_IF', 'ASSIGN_EXPR'])

# An action descriptor is a tuple (line_no, action_no, action, Expressions)
LINE_NO    = 0
ACT_NO     = 1
ACT        = 2
VAR_NAME   = 3
EXPR       = 4

Mode = Enum('Mode', ['IN_PARAM', 'OUT_PARAM', 'IN_OUT_PARAM', 'IN_GLOBAL', 'IN_OUT_GLOBAL', 'LOCAL_RW', 'LOCAL_RO'])

vars        = {}
var_count   = 0
state_stack = [] # [Var_State, ...]
frame_ptr   = 0
frame       = 0

# traversal_condition (True/False, Condition)
traversal_condition = []
branch_stack = [] # [(action, traversal_condition, (last_frame_ptr, last_frame)), ...]
TOP         = -1
ACTION      = 0
T_COND      = 1
FRAME_PARAM = 2
PTR         = 0
SIZE        = 1


class Var_State:
    def __init__(self, name, mode, well_formed, scalar, bounds, invariant):
        self.name        = name
        self.mode        = mode
        self.well_formed = well_formed # (lb, ub) lb is true only when var is well-defined
                                       # on the current branch. ub is a list of traversal conditions
                                       # for the branches where the variable is-well defined
        self.scalar      = scalar      
        self.bounds      = bounds      # (lb, ub)
        self.invariant   = invariant
        self.evaluated   = False
        self.expr        = None
        self.assigned    = False
        self.updated     = False

    def print_var_state(self):
        print('name =', self.name)
        print('mode =', self.mode)
        print('well_formed =', self.well_formed)
        print('scalar =', self.scalar)
        print('bounds =', self.bounds)
        print('invariant =', self.invariant)
        print('evaluated =', self.evaluated)
        print('expr =', self.expr)
        print('assigned =', self.assigned)
        print('updated =', self.updated)
            
def print_state_stack(stack):
    for i in range(len(stack)):
        stack[i].print_var_state()
        print('----------------------')

def new_frame(from_frame):
    result = []
    for i in range (from_frame[PTR], from_frame[PTR] + from_frame[SIZE]):
        result = result + [copy.deepcopy(state_stack[i])]
        result[-1].updated = False
    return result

def merge(frame_1, frame_2):
    print("merging ", frame_1, " and ", frame_2)



In [78]:
class Get_Names(ast.NodeVisitor):
    def __init__(self):
        self.names = []

    def visit_Name(self, node):
        self.names = self.names + [node.id]
        self.generic_visit(node)

    def name_set(self):
        return set(self.names)

get_names=Get_Names()

In [ ]:
# Translate a procedure body declaration.
# At the moment only procedure bodies are considered.
# A procedure declaration starts on a new line (possibly preceded by white spaces).
# It may have a global specification.
# It may declare local variables
# The statement is considered to be syntactically correct.
def translate_procedure_body (line_num, statement):
    # Statement contains ':='
    [lhs, rhs] = statement.split(':=')
    rhs = rhs.strip()
    if '(' not in lhs:
        # Non-indexed assignment.
        assigned_var = lhs.strip()
        index = ''
        evaluated_vars = set()
    else:
        # Indexed assignment.
        [v, index] = lhs.split('(')
        assigned_var = v.strip()
        index = '(' + index
        # Currently SPARK assumes an update of an array references the entire array.
        evaluated_vars = {assigned_var}

    rhs_parsed = ast.parse(rhs, mode='eval')
    get_names.visit(rhs_parsed)
    evaluated_vars = evaluated_vars | get_names.name_set()

    if index != '':
        # The index of the assigned variable is evaluated.
        get_names.visit(ast.parse(index, mode='eval'))
        evaluated_vars = get_names.name_set()

    return [(line_num, 1, Action.EVAL, list(evaluated_vars), None),
              (line_num, 2, Action.ASSIGN, [assigned_var], (rhs, rhs_parsed))]

In [ ]:
# Translate an assignment statement
# Assumes each line contains at most one statement.
# A statement may span more than one line but is identified by
# the first line number on which it occurs.
# The statement is considered to be syntactically correct.
def translate_assign (line_num, statement):
    # Statement contains ':='
    [lhs, rhs] = statement.split(':=')
    rhs = rhs.strip()
    if '(' not in lhs:
        # Non-indexed assignment.
        assigned_var = lhs.strip()
        index = ''
        evaluated_vars = set()
    else:
        # Indexed assignment.
        [v, index] = lhs.split('(')
        assigned_var = v.strip()
        index = '(' + index
        # Currently SPARK assumes an update of an array references the entire array.
        evaluated_vars = {assigned_var}

    rhs_parsed = ast.parse(rhs, mode='eval')
    get_names.visit(rhs_parsed)
    evaluated_vars = evaluated_vars | get_names.name_set()

    if index != '':
        # The index of the assigned variable is evaluated.
        get_names.visit(ast.parse(index, mode='eval'))
        evaluated_vars = get_names.name_set()

    return [(line_num, 1, Action.EVAL, list(evaluated_vars), None),
              (line_num, 2, Action.ASSIGN, [assigned_var], (rhs, rhs_parsed))]


In [83]:
# Translate a statement
# Assumes each line contains at most one statement.
# A statement may span more than one line but is identified by
# the first line number on which it occurs.
# The statement is considered to be syntactically correct.
def translate_assign (line_num, statement):
    # Statement contains ':='
        # Assignment statement.
        [lhs, rhs] = statement.split(':=')
        rhs = rhs.strip()
        if '(' not in lhs:
            # Non-indexed assignment.
            assigned_var = lhs.strip()
            index = ''
            evaluated_vars = set()
        else:
            # Indexed assignment.
            [v, index] = lhs.split('(')
            assigned_var = v.strip()
            index = '(' + index
            # Currently SPARK assumes an update of an array references the entire array.
            evaluated_vars = {assigned_var}

        rhs_parsed = ast.parse(rhs, mode='eval')
        get_names.visit(rhs_parsed)
        evaluated_vars = evaluated_vars | get_names.name_set()

        if index != '':
            # The index of the assigned variable is evaluated.
            get_names.visit(ast.parse(index, mode='eval'))
            evaluated_vars = get_names.name_set()

        return [(line_num, 1, Action.EVAL, list(evaluated_vars), None),
                  (line_num, 2, Action.ASSIGN, [assigned_var], (rhs, rhs_parsed))]
        

In [84]:
translate_assign(1, 'w(z) := x + y - 10')

[(1, 1, <Action.EVAL: 12>, ['x', 'z', 'y'], None),
 (1,
  2,
  <Action.ASSIGN: 13>,
  ['w'],
  ('x + y - 10', <ast.Expression at 0x10a190250>))]

In [56]:
s

In [103]:
# Merging rules:
    #              Var-A-lb  Var-A-ub  Var-B-lb  Var-B-ub   Merged
    # well_formed   False    []        False     []         (False, [])
    #               False    NE_List   False     []         (False, NE_List)
    #               True     []        False     List       (True, [])
    #               True     NE_List   False     []         (False, NE_List)
    #               False    NE_List-A False     NE_List-B  (False, NE_List-A + NE_List-B)
    #               True     []        True      []         (True, [])
    #               True     NE_List-A True      NE_List-B  (True, NE_List-A + NE_List-B)
        
def merge_well_formed(value_a, value_b):
    if value_a == value_b:
        # No change if the values are the same.
        result = value_a
    elif value_a == (True, []) or value_b == (True, []):
        # The variable has been assigned a value at the outermost level of the subprogram.
        # It will be well-formed from this point on.
        result = (True, [])
    else:
        (lb_a, ub_a) = value_a
        (lb_b, ub_b) = value_b
        if ub_a == ub_b:
            merged_traversal = ub_a
        else:
            merged_traversal = ub_a + ub_b
        result = (lb_a and lb_b, merged_traversal)
    return result
        

In [104]:
merge_well_formed((False, [123]), (True, [456]))

(False, [123, 456])

In [105]:
print_state_stack(state_stack)

In [106]:
# Each statement is a list of actions on variables
def process_statement (actions):
    global vars, var_count, state_stack, frame_ptr, frame, branch_stack, branch_ptr, traversal_condition
    for act in actions:
        if act[ACT] == Action.PROC_DEC:
            frame_ptr = frame_ptr + frame
        elif act[ACT] == Action.IN_PARAM:
            state_stack = state_stack + [Var_State(name=act[VAR_NAME], 
                                                   mode=Mode.IN_PARAM, 
                                                   well_formed=(True, []),
                                                   scalar=True, 
                                                   bounds=(Min_Int, Max_Int),
                                                   invariant=True)]
            frame = frame + 1
            vars[act[VAR_NAME]] = var_count
            var_count = var_count + 1
        elif act[ACT] == Action.OUT_PARAM:
            state_stack = state_stack + [Var_State(name=act[VAR_NAME], 
                                                   mode=Mode.OUT_PARAM, 
                                                   well_formed=(False, []), 
                                                   scalar=True,
                                                   bounds=(None, None),
                                                   invariant=False)]
            frame = frame + 1
            vars[act[VAR_NAME]] = var_count
            var_count = var_count + 1
        elif act[ACT] == Action.IN_OUT_PARAM:
            state_stack = state_stack + [Var_State(name=act[VAR_NAME], 
                                                   mode=Mode.IN_OUT_PARAM, 
                                                   well_formed=(True, []), 
                                                   scalar=True, 
                                                   bounds=(Min_Int, Max_Int),
                                                   invariant=False)]
            frame = frame + 1        
            vars[act[VAR_NAME]] = var_count
            var_count = var_count + 1
        elif act[ACT] == Action.IN_GLOBAL:
            state_stack = state_stack + [Var_State(name=act[VAR_NAME], 
                                                   mode=Mode.IN_GLOBAL, 
                                                   well_formed=(True, []),
                                                   scalar=True, 
                                                   bounds=(Min_Int, Max_Int),
                                                   invariant=True)]
        elif act[ACT] == Action.OUT_GLOBAL:
            state_stack = state_stack + [Var_State(name=act[VAR_NAME], 
                                                   mode=Mode.OUT_GLOBAL, 
                                                   well_formed=(False, []), 
                                                   scalar=True, 
                                                   bounds=(None, None),
                                                   invariant=False)]
            frame = frame + 1
            vars[act[VAR_NAME]] = var_count
            var_count = var_count + 1
        elif act[ACT] == Action.IN_OUT_GLOBAL:
            state_stack = state_stack + [Var_State(name=act[VAR_NAME], 
                                                   mode=Mode.IN_OUT_GLOBAL, 
                                                   well_formed=(True, []),
                                                   scalar=True, 
                                                   bounds=(Min_Int, Max_Int),
                                                   invariant=False)]
            frame = frame + 1        
            vars[act[VAR_NAME]] = var_count
            var_count = var_count + 1
        elif act[ACT] == Action.LOCAL_RW:
            state_stack = state_stack + [Var_State(name=act[VAR_NAME], 
                                                   mode=Mode.LOCAL_RW, 
                                                   well_formed=(False, []), 
                                                   scalar=True, 
                                                   bounds=(None, None),
                                                   invariant=False)]
            frame = frame + 1
            vars[act[VAR_NAME]] = var_count
            var_count = var_count + 1
        elif act[ACT] == Action.LOCAL_RO:
            state_stack = state_stack + [Var_State(name=act[VAR_NAME], 
                                                   mode=Mode.LOCAL_RO, 
                                                   well_formed=(False, []), 
                                                   scalar=True, 
                                                   bounds=(None, None),
                                                   invariant=False)]
            frame = frame + 1
            vars[act[VAR_NAME]] = var_count
            var_count = var_count + 1
        elif act[ACT] == Action.EVAL:
            state_stack[vars[act[VAR_NAME]] + frame_ptr].evaluated=True
        elif act[ACT] == Action.ASSIGN_EXPR:
            state_stack[vars[act[VAR_NAME]] + frame_ptr].expr=ast.parse(act[EXPR], mode='eval')
        elif act[ACT] == Action.ASSIGN:
            state_index = vars[act[VAR_NAME]] + frame_ptr
            if state_stack[state_index].assigned:
                state_stack[state_index].updated = True
            else:
                state_stack[state_index].assigned = True

            state_stack[state_index].well_formed = merge_well_formed(
                state_stack[state_index].well_formed,
                # The variable is well-formed on this branch following the assignment.
                # Copy add the branch traversal condition.
                (True, copy.deepcopy(traversal_condition)))
        elif act[ACT] == Action.IF_COND:
            # The start of a compound statement.
            # Save the current traversal condition and state_stack parameters onto the branch_stack.
            branch_stack = branch_stack + [(Action.IF_COND, 
                                            copy.deepcopy(traversal_condition), (frame_ptr, frame))]
            # Start a new stack frame for the branch. It is the same as the frame prior to the IF_COND
            state_stack = state_stack + new_frame((frame_ptr, frame))
            frame_ptr = frame_ptr + frame
            # Add the condition of the IF_COND to the traversal condition
            # The current traversal condition is extended to include the IF_COND
            if traversal_condition == []:
                traversal_condition =  [(True, ast.parse(act[EXPR], mode='eval'))]
            else:
                traversal_condition = traversal_condition + [(True, 
                                                              ast.parse(act[EXPR], mode='eval'))]

            # May be able to adjust bounds of variables used in condition.
            # for instance, if condition is 'x > y + 10' then x lower bound is lb(y) + 10
        elif act[ACT] == Action.ELSE_COND:
            # Check whether the previous branch was the IF branch of the IF statement.
            if branch_stack[TOP][ACTION] == Action.IF_COND:
                # Then save the traversal condition and state_stack frame parameters on the branch_stack
                branch_stack = branch_stack + [(Action.ELSE_COND,
                                                traversal_condition, (frame_ptr, frame))]
            else:
                # It is not an IF branch, merge the last two branches.
                # The resulting merged state is in the lower of the two state_stack frames.
                merge((branch_stack[TOP-1][FRAME_PARAM]), (branch_stack[TOP][FRAME_PARAM]))
                # Pop off the top state_stack frame.
                frame_ptr = frame_ptr - frame                
                state_stack = state_stack[:frame_ptr+frame]

            # Start a new state_stack frame from the IF_COND frame.
            # The IF_COND frame parameters are always available from the penultimate branch_stack
            # entry as all ELSE_COND branches are merged and have the same branch_stack entry
            state_stack = state_stack + new_frame(branch_stack[TOP-1][FRAME_PARAM])
            frame_ptr = frame_ptr + frame
            
            # The traversal for this branch is the same as the previous IF_COND or ELSE_COND branch
            # but with the IF_COND or ELSE_COND negated and traversal condition extend 
            # with the condition of this ELSE_COND.

            # Set the last condition in the traversal condition FALSE
            traversal_condition [-1] = (False, traversal_condition[-1][1])
            # Now extend the traversal condition with the ELSE_COND.
            traversal_condition = traversal_condition + [(True, 
                                                          ast.parse(act[EXPR], mode='eval'))]
        elif act[ACT] == Action.ELSE_BRANCH:
            # Check whether the previous branch was the IF branch of the IF statement.
            if branch_stack[TOP][ACTION] != Action.IF_COND:
                # It is not an IF branch, merge the last two branches
                # The resulting merged state is in the lower of the two state_stack frames.
                merge((branch_stack[TOP-1][FRAME_PARAM]), (branch_stack[TOP][FRAME_PARAM]))
                # Pop off the top state_stack frame.
                frame_ptr = frame_ptr - frame
                state_stack = state_stack[:frame_ptr+frame]
                # Pop off the ELSE_COND from the branch_stack
                branch_stack = branch_stack[:len(branch_stack)-1]
            
            # Save the traversal condition and current stack frame parameters on the branch_stack
            # and start a new stack frame for the ELSE branch.
            branch_stack = branch_stack + [(Action.ELSE_BRANCH,
                                            traversal_condition, (frame_ptr, frame))]
            state_stack = state_stack + new_frame((frame_ptr, frame))
            frame_ptr = frame_ptr + frame
            # Set the last condition in the traversal condition FALSE
            traversal_condition [-1] = (False, traversal_condition[-1][1])
            # The traversal_condition is not extended for an ELSE branch
        elif act[ACT] == Action.END_IF:
            # Check whether there is not an ELSE branch
            if branch_stack[TOP][ACTION] != Action.ELSE_BRANCH:
                # There is no ELSE branch. the merged branches of the IF statement
                # have to be merged with the state preceding the IF_COND
                
                # Check whether the previous branch is not an IF branch of the IF statement.
                if branch_stack[TOP][ACTION] != Action.IF_COND:
                    # It is not an IF branch, merge the last two branches
                    # The resulting merged state is in the lower of the two state_stack frames.
                    # That is the IF_COND state frame
                    merge((branch_stack[TOP-1][FRAME_PARAM]), (branch_stack[TOP][FRAME_PARAM]))
                    # Pop off the top state_stack frame.
                    frame_ptr = frame_ptr - frame
                    state_stack = state_stack[:frame_ptr+frame]
                    # Pop off the ELSE_COND from the branch_stack
                    branch_stack = branch_stack[:len(branch_stack)-1]
                    
                # Now merge the IF_COND state frame with the frame preceding the IF_COND  
                # The resulting merged state is in the state frame preceding the IF_COND.
                merge((branch_stack[TOP-1][FRAME_PARAM]), (branch_stack[TOP][FRAME_PARAM]))
                # Pop off the top state_stack frame.
                frame_ptr = frame_ptr - frame
                state_stack = state_stack[:frame_ptr+frame] 
                # Pop off the IF_COND from the branch_stack
                branch_stack = branch_stack[:len(branch_stack)-1]
            else:
                # There is an ELSE branch.  The merged branches of the IF-THEN-ELSE statement
                # is the state that is carried forward in the analysis.
                # The state prior to the IF_COND has been accounted for in each branch
                # of the IF_THEN_ELSE and is no longer required.

                # Merge the state ELSE_BRANCH and the previous accumulated states in the IF_COND branch.
                # The result of the merge is in the state of the IF_COND branch.
                merge((branch_stack[TOP-1][FRAME_PARAM]), (branch_stack[TOP][FRAME_PARAM]))
                # Pop off the top state_stack frame.
                frame_ptr = frame_ptr - frame
                state_stack = state_stack[:frame_ptr+frame]
                # Pop off the ELSE_BRANCH from the branch_stack
                branch_stack = branch_stack[:len(branch_stack)-1]

                # Replace the state prior to the IF_COND with the state from the IF_COND branch and
                # pop off IF_COND branch state
                (pre_ptr, pre_frame) = branch_stack[TOP][FRAME_PARAM]
                state_stack = state_stack[:pre_ptr] + state_stack[frame_ptr:frame_ptr+frame]
                frame_ptr = pre_ptr  # The IF statement may have introduced new variables,
                                     # although no longer in scope, they have been added to
                                     # the abstract state and therefore the frame count is unchanged.

                # Pop off IF_COND from the branch_stack.
                branch_stack = branch_stack[:len(branch_stack)-1]
        else:
            print("Unsupported ", act[ACT]) 
                    

In [107]:
process_statement([(1, 1, Action.PROC_DEC, 'p', None), 
                   (1, 2, Action.IN_OUT_PARAM, 'x', None),
                   (1, 3, Action.IN_OUT_PARAM, 'y', None)])
print_state_stack(state_stack)

name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = False
expr = None
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = False
expr = None
assigned = False
updated = False
----------------------


In [108]:
vars

{'x': 0, 'y': 1}

In [109]:
process_statement([(2, 1, Action.LOCAL_RW, 'temp', None)])
print_state_stack(state_stack)               

name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = False
expr = None
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = False
expr = None
assigned = False
updated = False
----------------------
name = temp
mode = Mode.LOCAL_RW
well_formed = (False, [])
scalar = True
bounds = (None, None)
invariant = False
evaluated = False
expr = None
assigned = False
updated = False
----------------------


In [110]:
process_statement([(3, 1, Action.EVAL, 'x', None),
                  (3, 2, Action.EVAL, 'y', None),
                  (3, 3, Action.IF_COND, None, 'x > y + 10')])
print_state_stack(state_stack)

name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = True
expr = None
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = True
expr = None
assigned = False
updated = False
----------------------
name = temp
mode = Mode.LOCAL_RW
well_formed = (False, [])
scalar = True
bounds = (None, None)
invariant = False
evaluated = False
expr = None
assigned = False
updated = False
----------------------
name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = True
expr = None
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = True
expr = None
assigned = Fa

In [111]:
branch_stack

[(<Action.IF_COND: 16>, [], (0, 3))]

In [112]:
traversal_condition

[(True, <ast.Expression at 0x10ac5e510>)]

In [113]:
vars

{'x': 0, 'y': 1, 'temp': 2}

In [114]:
process_statement([(4, 1, Action.EVAL,'x', None), 
                   (4, 2, Action.ASSIGN_EXPR, 'temp', 'x'),
                   (4, 3, Action.ASSIGN, 'temp', None)])

In [115]:
print_state_stack(state_stack)

name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = True
expr = None
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = True
expr = None
assigned = False
updated = False
----------------------
name = temp
mode = Mode.LOCAL_RW
well_formed = (False, [])
scalar = True
bounds = (None, None)
invariant = False
evaluated = False
expr = None
assigned = False
updated = False
----------------------
name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = True
expr = None
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = True
expr = None
assigned = Fa

In [116]:
process_statement([(5, 1, Action.EVAL, 'y', None),
                   (5, 2, Action.ASSIGN_EXPR, 'x', 'y'),
                   (5, 3, Action.ASSIGN, 'x', None)])

In [117]:
print_state_stack(state_stack)

name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = True
expr = None
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = True
expr = None
assigned = False
updated = False
----------------------
name = temp
mode = Mode.LOCAL_RW
well_formed = (False, [])
scalar = True
bounds = (None, None)
invariant = False
evaluated = False
expr = None
assigned = False
updated = False
----------------------
name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = True
expr = <ast.Expression object at 0x10acedd10>
assigned = True
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated

In [118]:
process_statement([(6, 1, Action.EVAL,'temp', None),
                   (6, 2, Action.ASSIGN_EXPR, 'y', 'temp'),
                   (6, 3, Action.ASSIGN, 'y', None)])

In [119]:
print_state_stack(state_stack)

name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = True
expr = None
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = True
expr = None
assigned = False
updated = False
----------------------
name = temp
mode = Mode.LOCAL_RW
well_formed = (False, [])
scalar = True
bounds = (None, None)
invariant = False
evaluated = False
expr = None
assigned = False
updated = False
----------------------
name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = True
expr = <ast.Expression object at 0x10acedd10>
assigned = True
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated

In [120]:
process_statement([(7, 1, Action.EVAL, 'x', None), 
                   (7, 2, Action.EVAL, 'y', None), 
                   (7, 3, Action.ELSE_COND, None, 'x > y')])

In [121]:
traversal_condition

[(False, <ast.Expression at 0x10ac5e510>),
 (True, <ast.Expression at 0x10b620610>)]

In [122]:
branch_stack


[(<Action.IF_COND: 16>, [], (0, 3)),
 (<Action.ELSE_COND: 17>, [(False, <ast.Expression at 0x10ac5e510>)], (3, 3))]

In [123]:
traversal_condition

[(False, <ast.Expression at 0x10ac5e510>),
 (True, <ast.Expression at 0x10b620610>)]

In [124]:
print_state_stack(state_stack)

name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = True
expr = None
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = True
expr = None
assigned = False
updated = False
----------------------
name = temp
mode = Mode.LOCAL_RW
well_formed = (False, [])
scalar = True
bounds = (None, None)
invariant = False
evaluated = False
expr = None
assigned = False
updated = False
----------------------
name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = True
expr = <ast.Expression object at 0x10acedd10>
assigned = True
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated

In [126]:
 process_statement([(8, 1, Action.EVAL,'x',  None), 
                    (8, 3, Action.ASSIGN_EXPR, 'y', 'x + 10'),
                    (8, 4, Action.ASSIGN, 'y', None)]) 

In [127]:
print_state_stack(state_stack)

name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = True
expr = None
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = True
expr = None
assigned = False
updated = False
----------------------
name = temp
mode = Mode.LOCAL_RW
well_formed = (False, [])
scalar = True
bounds = (None, None)
invariant = False
evaluated = False
expr = None
assigned = False
updated = False
----------------------
name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = True
expr = <ast.Expression object at 0x10acedd10>
assigned = True
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated

In [238]:
process_statement([(Action.ELSE_BRANCH, '')])

merging  (0, 3)  and  (3, 3)


In [239]:
branch_stack

[(<Action.IF_COND: 16>, [], (0, 3)),
 (<Action.ELSE_BRANCH: 18>,
  [(False,
    "Expression(body=Compare(left=Name(id='x', ctx=Load()), ops=[Gt()], comparators=[BinOp(left=Name(id='y', ctx=Load()), op=Add(), right=Constant(value=10))]))"),
   (False,
    "Expression(body=Compare(left=Name(id='x', ctx=Load()), ops=[Gt()], comparators=[Name(id='y', ctx=Load())]))")],
  (3, 3))]

In [240]:
process_statement([(Action.END_IF, '')])

merging  (0, 3)  and  (3, 3)


In [241]:
print(branch_stack)

[]


In [242]:
print_state_stack(state_stack)

name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = True
assigned = True
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
bounds = (-2147483648, 2147483647)
invariant = False
evaluated = True
assigned = True
updated = False
----------------------
name = temp
mode = Mode.LOCAL_RW
well_formed = (True, [(True, "Expression(body=Compare(left=Name(id='x', ctx=Load()), ops=[Gt()], comparators=[BinOp(left=Name(id='y', ctx=Load()), op=Add(), right=Constant(value=10))]))")])
scalar = True
bounds = (None, None)
invariant = False
evaluated = True
assigned = True
updated = False
----------------------


In [243]:
print(frame_ptr, frame)

0 3


In [2]:
import ast
for node in ast.walk(ast.parse('x')):
    print(node)
    print(node.__dict__)
    print("children: " + str([x for x in ast.iter_child_nodes(node)]) + "\\n")

{'body': [<ast.Expr object at 0x1098b0bd0>], 'type_ignores': []}
children: [<ast.Expr object at 0x1098b0bd0>]\n
{'value': <ast.Name object at 0x1098b0250>, 'lineno': 1, 'col_offset': 0, 'end_lineno': 1, 'end_col_offset': 1}
children: [<ast.Name object at 0x1098b0250>]\n
{'id': 'x', 'ctx': <ast.Load object at 0x105dd4610>, 'lineno': 1, 'col_offset': 0, 'end_lineno': 1, 'end_col_offset': 1}
children: [<ast.Load object at 0x105dd4610>]\n
{'parent': <ast.Name object at 0x10a0bc9d0>}
children: []\n


In [18]:
import ast
from pprint import pprint

class Analyzer(ast.NodeVisitor):
    def __init__(self):
        self.stats = {"expr": [], "name": []}

    def visit_Expression(self, node):
        self.stats["expr"].append(node.body)
        self.generic_visit(node)

    def visit_Name(self, node):
        self.stats["name"].append(node.id)
        self.generic_visit(node)

    def report(self):
        pprint(self.stats)


In [28]:
analyzer = Analyzer()
tree = ast.parse ('y * 10', mode='eval')
analyzer.visit(tree)
analyzer.report()

{'expr': [<ast.BinOp object at 0x10ac6e4d0>], 'name': ['y']}


In [22]:
ast.dump(tree)

"Module(body=[Assign(targets=[Name(id='x', ctx=Store())], value=BinOp(left=Name(id='y', ctx=Load()), op=Mult(), right=Constant(value=10)))], type_ignores=[])"

In [7]:
import ast
ast.dump(ast.parse('a[b]', mode='eval'))

"Expression(body=Subscript(value=Name(id='a', ctx=Load()), slice=Name(id='b', ctx=Load()), ctx=Load()))"

In [13]:
':=' in 'a = b'

False

In [17]:
'a'.split('(')

['a']

In [21]:
ast.dump(ast.parse('(10)', mode='eval'))

'Expression(body=Constant(value=10))'

In [34]:
class Get_Names(ast.NodeVisitor):
    def __init__(self):
        self.names = []

    def visit_Name(self, node):
        self.names = self.names + [node.id]
        self.generic_visit(node)

    def name_list(self):
        return self.names


In [37]:
evaluated_names=Get_Names()
evaluated_names.visit(ast.parse("x + y - 10 * (z - n)", mode='eval'))

In [36]:
ast.dump(ast.parse("x + y - 10 * (z - n)", mode='eval'))

"Expression(body=BinOp(left=BinOp(left=Name(id='x', ctx=Load()), op=Add(), right=Name(id='y', ctx=Load())), op=Sub(), right=BinOp(left=Constant(value=10), op=Mult(), right=BinOp(left=Name(id='z', ctx=Load()), op=Sub(), right=Name(id='n', ctx=Load())))))"

In [38]:
evaluated_names.name_list()

['x', 'y', 'z', 'n']

In [86]:
c='''
Now is the 
time for all good
men to come to'''

In [87]:
c


'\nNow is the \ntime for all good\nmen to come to'